# Import required Libraries

We will be using Pandas to analyze and query the GBIF dataset. Regular Expressions allows us to query with a high-degree of variance.

In [1]:
import pandas as pd
import re

In [2]:
pd.__version__

'1.3.5'

# Load GBIF Data

Make sure to place the gbif.csv file in the data folder. Also, name it gbif.csv.

In [ ]:
!wc -l occurrence.txt

In [ ]:
df = pd.read_csv("occurrence.txt", 
                 sep="\t", on_bad_lines='skip',
                 nrows=100000)
df.info()

In [ ]:
list(df.columns)

In [ ]:
restrict_columns = ["identifiedBy", 
                    "recordedBy", 
                    "occurrenceID"]
df = pd.read_csv("/scratch/genomics/dikowr/AWHI-collections-data-funk_list/data/occurrence.txt", 
                 sep="\t", on_bad_lines='skip',
                 usecols=restrict_columns,
                 nrows=100000)
df.info()

In [ ]:
restrict_columns = ["identifiedBy", 
                    "recordedBy", 
                    "occurrenceID"]
df = pd.read_csv("/scratch/genomics/dikowr/AWHI-collections-data-funk_list/data/occurrence.txt", 
                 sep="\t", on_bad_lines='skip',
                 usecols=restrict_columns)
df.info()

# Load Funk List Data

In [ ]:
funk_df = pd.read_csv("Funk_List_Names_RegEx.csv")
funk_df = funk_df.fillna(False)
funk_df

# Reverse the Formulas for identifiedBy

In [9]:
reverse_names = []
reverse_spouse_names = []
for index, row in funk_df.iterrows():
    lastname = row.last_name
    first_name = row.first_name
    middle_name = row.middle_name
    if middle_name != False:
    # Funk, (V(\.|icki)* )(A(\.|nn)*)*)
        reverse_names.append(f"{lastname}, {first_name[0]}(\.|{first_name[1:]})(( *{middle_name[0]})(\.|{middle_name[1:]})*)*")
    else:
        if first_name != False:
            reverse_names.append(f"{lastname}, {first_name[0]}(\.|{first_name[1:]})")
        else:
            print(lastname, first_name)
    if row.SPOUSE != False:
        spouse_names = row.SPOUSE.split()
        spouse_lastname = spouse_names[-1]
        spouse_firstname = spouse_names[0]
        if len(spouse_names) == 3:
            spouse_middlename = spouse_names[1]
            reverse_spouse_names.append(f"{spouse_lastname}, {spouse_firstname[0]}(\.|{spouse_firstname[1:]})(( *{spouse_middlename[0]})(\.|{spouse_middlename[1:]})*)*")
        else:
            reverse_spouse_names.append(f"{spouse_lastname}, {spouse_firstname[0]}(\.|{spouse_firstname[1:]})")
    else:
        reverse_spouse_names.append(False)
funk_df["regex_reverse"] = reverse_names
funk_df["regex_reverse_spouse"] = reverse_spouse_names

# Extract the Data

In [10]:
import os

In [11]:
def extract_results(df,
                    query_columns: list,
                    regex_formula:str,
                    directory: str,
                    woman:str,
                    variant: bool,
                    spouse: bool,
                    save_if_match=True
                   ):
    """
    This function will take an input DataFrame and match a RegEx formula on a specific column in the DataFrame.
    - df = DataFrame (GBIF)
    - query_columns = the columns in the GBIF dataset that you wish to query
    - regex_formula = the formula you seek to match
    - output_file = the file to which you wish to save the results
    - woman = woman's name
    - spouse = Boolean - if it is a spouse, set this to True
    - save_if_match (optional) - if True, it only saves when matches are found
    """
    for query_column in query_columns:
        res = df.loc[df[query_column].str.contains(regex_formula, regex=True).fillna(False)]
        if save_if_match==True and len(res) > 0:
            path = f"{directory}/{query_column}"
            if spouse==True:
                path=path+"_spouse"
            if variant == True:
                path = path+"_variant"
            if os.path.exists(path):
                pass
            else:
                os.mkdir(path)
                
            output_file = f"{path}/{woman}.csv"
            res.to_csv(output_file, index=False)
    return res

In [ ]:
for index, row in funk_df.iterrows():
    woman = f"{row['last_name']}_{row['first_name']}"
    
    ### FUNKLIST WOMAN
    regex_formula = row["regex_formulas"]
    try:
        extract_results(df, ["identifiedBy", "recordedBy"], regex_formula, "gbif_data", woman, variant=False, spouse=False)
    except:
        TypeError
    
    regex_formula_variant = row["regex_formulas_variants"]
    try:
        extract_results(df, ["identifiedBy", "recordedBy"], regex_formula_variant, "gbif_data", woman, variant=True, spouse=False)
    except:
        TypeError
    
    regex_formula_reverse = row["regex_reverse"]
    try:
        extract_results(df, ["identifiedBy"], regex_formula_reverse, "gbif_data", woman, variant=False, spouse=False)
    except:
        TypeError
    
    
    ### SPOUSES
    regex_formula_spouse = row["regex_formulas_spouse"]
    try:
        extract_results(df, ["identifiedBy", "recordedBy"], regex_formula_spouse, "gbif_data", woman, variant=False, spouse=True)
    except:
        TypeError
    
    
    regex_formula_spouse_reverse = row["regex_reverse_spouse"]
    try:
        extract_results(df, ["identifiedBy"], regex_formula_spouse_reverse, "gbif_data", woman, variant=False, spouse=True)
    except:
        TypeError
    
